# <center>Laboratorium Analiza i bazy danych </center>

## <center>Łączenie tabel, podzapytania i funkcje agregujące</center>

## Przykładowe tabele obrazujące łączenie

Do zobrazowania operacji łączenia zostaną użyte tabele:

```sql
CREATE TABLE shape_a (
    id INT PRIMARY KEY,
    shape VARCHAR (100) NOT NULL
);
 
CREATE TABLE shape_b (
    id INT PRIMARY KEY,
    shape VARCHAR (100) NOT NULL
);
```
 
Polecenie CREATE TABLE tworzy tabelę o zadanej nazwie i strukturze. Ogólna postać to:
```sql
CREATE TABLE tab_name (
    col_name1 data_type constrain,
    col_name1 data_type constrain,
    ...
);
```
Należy uzupełnić ją danymi:
```sql
INSERT INTO shape_a (id, shape)
VALUES
    (1, 'Trójkąt'),
    (2, 'Kwadrat'),
    (3, 'Deltoid'),
    (4, 'Traper');
 
INSERT INTO shape_b (id, shape)
VALUES
    (1, 'Kwadrat'),
    (2, 'Trójkąt'),
    (3, 'Romb'),
    (4, 'Równoległobok');
```
Komenda INSERT INTO pozwala na dodanie do tabeli rekordów. Ogólna postać to:

```sql
INSERT INTO tab_name (col1_name, col2_name2, ...) 
VALUES
    (val1_col1, val2_col2),
    (val2_col1, val2_col2),
    ...
```

## Inner join 

Jest to podstawowy rodzaj złączenie. Ten sposób złączenia wybiera  te wiersze, dla których warunek złączenia jest spełniony. W żadnej z łączonych tabel kolumna użyta do łączenia nie może mieć wartości NULL. 

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
INNER JOIN shape_b b ON a.shape = b.shape;
```
W zapytaniu powyżej użyto *aliasów* nazw tabel i column wynikowych, jest to szczególnie przydatne przy długich nazwach tabel i wprowadza czytelność w zapytaniu.

#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|

## OUTER JOIN

Istnieją trzy rodzaje złączeń OUTER:
- LEFT OUTER JOIN,
- RIGHT OUTER JOIN,
- FULL OUTER JOIN.

### LEFT OUTER JOIN

Ten rodzaj złączenie zwróci wszystkie rekordy z lewej tablicy i dopasuje do nich rekordy z prawej tablicy które spełniją zadany warunek złączenia. Jeżeli w prawej tablicy nie występują rekordy spełnijące warunek złączenia z lewą w ich miejscu pojawią się wartości NULL.

#### Przykład 1:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
LEFT JOIN shape_b b ON a.shape = b.shape;
```
#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|
|3|Deltoid|NULL|NULL|
|4|Traper|NULL|NULL|

#### Przykład 2:
```sql
SELECT
    b.id id_b,
    b.shape shape_b,
    a.id id_a,
    a.shape shape_a   
FROM
    shape_b b
LEFT JOIN shape_a a ON a.shape = b.shape;
```
#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Kwadrat|2|Kwadrat|
|2|Trójkąt|1|Trójkąt|
|3|Romb|NULL|NULL|
|4|Równoległobok|NULL|NULL|

### RIGHT OUTER JOIN

Działa jak left outer join z tym, że prawa tablica w zapytaniu jest brana w całości.

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
RIGHT JOIN shape_b b ON a.shape = b.shape;
```

#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|2|Kwadrat|1|Kwadrat|
|1|Trójkąt|2|Trójkąt|
|NULL|NULL|3|Romb|
|NULL|NULL|4|Równoległobok|


### FULL OUTER JOIN

Jest złączeniem które zwraca:
- wiersze dla których warunek złączenia jest spełniony,
- wiersze z lewej tabeli dla których nie ma odpowiedników w prawej,
- wiersze z prawej tabeli dla których nie ma odpowiedników w lewej. 

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
FULL JOIN shape_b b ON a.shape = b.shape;
```
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|
|3|Deltoid"|NULL|NULL|
|4|Traper|NULL|NULL|
|NULL|NULL|3|Romb|
|NULL|NULL|4|Równoległobok|

## Podzapytania

Podzapytanie zagnieżdżone SELECT znajduje się wewnątrz zewnętrznego zapytania SELECT, np. po klauzuli WHERE, HAVING lub FROM. W przypadku tego rodzaju zapytań w pierwszej kolejności wykonywane są wewnętrzne zapytania SELECT, a ich wynik jest wykorzystywany do zewnętrznego zapytania SELECT. Stąd łatwo zuważyć, że mogą one służyć do poprawy wydajności obsługi zapytania. Należy dobierać podzapytania tak by najbardziej zagnieżdżone podzapytanie zawierało najmniejszy zbiór poszukiwań. 

#### Przykład:
Jeżeli chcemy znaleźć w bazie informację o tytułach filmów zwróconych w zadanym okresie możemy wykonać następujące zapytanie:
```sql
SELECT
   film_id,
   title
FROM
   film
WHERE
   film_id IN (
      SELECT
         inventory.film_id
      FROM
         rental
      INNER JOIN inventory ON inventory.inventory_id = rental.inventory_id
      WHERE
         return_date BETWEEN '2005-05-29'
      AND '2005-05-30'
   );
```

#### Wynik
|film_id|title|
|-|-|
|307|Fellowship Autumn|
|255|Driving Polish|
|388|Gunfight Moon|
|130|Celebrity Horn|
|563|Massacre Usual|
|397|Hanky October|
|...|...|

### Używanie podzapytań

Pod zapytania mogą być używane w :
- SELECT,
- UPDATE,
- DELETE,
- Funkcjach agregujących,
- Do definiowania tabel tymczasowych.

Używając podzapytań zapytania SQL szybko mogą stać się mało czytelne. Przez co będą trudne w zrozumieniu i późniejszym utrzymaniu. W celu analizy zapytań można użyć klauzuli __EXPLAIN__, która przeanalizuje zapytanie. Klauzula ta może służyć również do porównywania wydajności zapytań

#### Przykład:
```sql
EXPLAIN SELECT
   *
FROM
   film
```

## Funkcje agregujące

Funkcje agregujące wykonują obliczenia na zestawie wierszy i zwracają pojedynczy wiersz. PostgreSQL udostępnia wszystkie standardowe funkcje agregujące SQL w następujący sposób:
- AVG () - zwraca średnią wartość.
- COUNT () - zwraca liczbę wartości.
- MAX () - zwraca maksymalną wartość.
- MIN () - zwraca minimalną wartość.
- SUM () - zwraca sumę wszystkich lub różnych wartości.

Pełna lista funkcji agregującej: https://www.postgresql.org/docs/9.5/functions-aggregate.html

Często używamy funkcji agregujących z klauzulą GROUP BY w instrukcji SELECT. W tych przypadkach klauzula GROUP BY dzieli zestaw wyników na grupy wierszy i funkcja agregująca wykonuje obliczenia dla każdej grupy, np. maksimum, minimum, średnia itp. Funkcji agregujących można używać funkcji agregujących jako wyrażeń tylko w następujących klauzulach: SELECT i HAVING.

### GROUP BY
Klauzula GROUP BY dzieli wiersze zwrócone z instrukcji SELECT na grupy. Dla  każdej grupy można zastosować funkcję agregującą, np. SUM aby obliczyć sumę pozycji lub
COUNT aby uzyskać liczbę elementów w grupach.

Poniższa instrukcja ilustruje składnię klauzuli GROUP BY:
```sql
SELECT 
    column_1, 
    aggregate_function(column_2)
FROM 
    tbl_name
GROUP BY 
    column_1;
```
Klauzula GROUP BY musi pojawić się zaraz po klauzuli FROM lub WHERE, n0astępnie GROUP BY zawiera listę  kolumna oddzielonych przecinkami. 

### HAVING
Często używamy klauzuli HAVING w połączeniu z klauzulą GROUP BY do filtrowania wierszy grup
które nie spełniają określonego warunku.

Poniższa instrukcja ilustruje typową składnię klauzuli HAVING:
```sql
SELECT
    column_1,
    aggregate_function (column_2)
FROM
    tbl_name
GROUP BY
    column_1
HAVING
    condition;
```
Klauzula HAVING ustawia warunek dla wierszy grup utworzonych przez klauzulę GROUP BY.  

Klauzula GROUP BY ma zastosowanie, podczas gdy klauzula WHERE określa wcześniej warunki dla poszczególnych wierszy.

## Zadania wprowadzające
Wykonaj zapytania przy użyciu DBMS:  
  
1. Znajdź liczbę wszystkich filmów o tej samej długości.
2. Znajdź wszystkich klientów mieszkających w tym samym mieście.
3. Oblicz średni koszt wypożyczenia wszystkich filmów.
4. Oblicz i wyświetl liczbę filmów we wszystkich kategoriach.
5. Wyświetl liczbę wszystkich klientów pogrupowanych według kraju.
6. Wyświetl informacje o sklepie, który ma więcej niż 100 klientów i mniej niż 300 klientów.
7. Wybierz wszystkich klientów, którzy oglądali filmy ponad 200 godzin.
8. Oblicz średnią wartość wypożyczenia filmu.
9. Oblicz średnią wartość długości filmu we wszystkich kategoriach.
10. Znajdź najdłuższe tytuły filmowe we wszystkich kategoriach.
11. Znajdź najdłuższy film we wszystkich kategoriach. Porównaj wynik z pkt 10.

## Zadanie implementacyjne
Zaimplementuj wszystkie funkcje w pliku main.py zgodnie z opisem a następnie przetestuj je w notatniku.

In [2]:
import main
import psycopg2 as pg
import pandas as pd

connection = pg.connect(host='pgsql-196447.vipserv.org', port=5432, dbname='wbauer_adb', user='wbauer_adb', password='adb2020')

Zadanie 1

In [3]:
l = 100
df = pd.read_sql("SELECT COUNT(*) FROM film WHERE film.length = {length}".format(length=l),con=connection)
df

,count
0,12


Zadanie 2

In [4]:
place = 'Moscow'
df = pd.read_sql("SELECT customer.first_name, customer.last_name, city.city FROM city "
                 "INNER JOIN address ON city.city_id = address.city_id " +
                 "INNER JOIN customer ON address.address_id = customer.address_id " + 
                 "WHERE city.city = '{x}'".format(x=place),con=connection)
df

,first_name,last_name,city
0,Catherine,Campbell,Moscow


In [5]:
#main.client_from_city('Moscow')

Zadanie 3

In [6]:
df = pd.read_sql("SELECT AVG(payment.amount) FROM film " + 
                 "INNER JOIN inventory ON film.film_id = inventory.film_id " + 
                 "INNER JOIN rental ON inventory.inventory_id = rental.inventory_id " +
                 "INNER JOIN payment ON rental.rental_id = payment.rental_id ", con=connection)
df

,avg
0,4.200606


Zadanie 4

In [7]:
df = pd.read_sql("SELECT category.name category, COUNT(category.name) FROM film " +
                 "INNER JOIN film_category ON film.film_id = film_category.film_id " +
                 "INNER JOIN category ON film_category.category_id = category.category_id " +
                 "GROUP BY category.name ", con=connection)
        
df

,category,count
0,Horror,56
1,Comedy,58
2,Sci-Fi,61
3,Drama,62
4,Foreign,73
5,Classics,57
6,Games,61
7,New,63
8,Travel,57
9,Music,51


Zadanie 5

In [8]:
df = pd.read_sql("SELECT country.country, COUNT(customer.customer_id) FROM country " +
                 "INNER JOIN city ON country.country_id = city.country_id " +
                 "INNER JOIN address ON city.city_id = address.city_id " +
                 "INNER JOIN customer ON address.address_id = customer.address_id " +
                 "GROUP BY country.country ORDER BY country.country", con=connection)
        
df

,country,count
0,Afghanistan,1
1,Algeria,3
2,American Samoa,1
3,Angola,2
4,Anguilla,1
...,...,...
103,Vietnam,6
104,"Virgin Islands, U.S.",1
105,Yemen,4
106,Yugoslavia,2


Zadanie 6

In [9]:
df = pd.read_sql("SELECT store.* FROM store " +
                 "INNER JOIN customer ON store.store_id = customer.store_id " +
                 "GROUP BY store.store_id " + 
                 "HAVING COUNT(customer.customer_id) > 100 AND COUNT(customer.customer_id) < 300", con=connection)
        
df

,store_id,manager_staff_id,address_id,last_update
0,2,2,2,2006-02-15 09:57:12


Zadanie 7

In [10]:
df = pd.read_sql("SELECT customer.first_name, customer.last_name, SUM(film.length) FROM film " +
                 "INNER JOIN inventory ON film.film_id = inventory.film_id " +
                 "INNER JOIN rental ON inventory.inventory_id = rental.inventory_id " +
                 "INNER JOIN customer ON rental.customer_id = customer.customer_id " +
                 "GROUP BY customer.first_name, customer.last_name " +
                 "HAVING SUM(film.length) > 200" +
                 "ORDER BY SUM(film.length), customer.last_name, customer.first_name", con=connection)

df

,first_name,last_name,sum
0,Brian,Wyman,1265
1,Antonio,Meek,1451
2,Leona,Obrien,1588
3,Katherine,Rivera,1615
4,Tiffany,Jordan,1667
...,...,...,...
594,Wesley,Bull,4808
595,Clara,Shaw,4808
596,Tammy,Sanders,5065
597,Eleanor,Hunt,5360


Zadanie 8

In [11]:
title_name = 'Youth Kick'
df = pd.read_sql("SELECT AVG(payment.amount) FROM film " + 
                 "INNER JOIN inventory ON film.film_id = inventory.film_id " + 
                 "INNER JOIN rental ON inventory.inventory_id = rental.inventory_id " +
                 "INNER JOIN payment ON rental.rental_id = payment.rental_id " +
                 "WHERE film.title = '{x}'".format(x = title_name), con=connection)
df

,avg
0,2.823333


Zadanie 9

In [12]:
df = pd.read_sql("SELECT category.name category, AVG(film.length) FROM film " +
                 "INNER JOIN film_category ON film.film_id = film_category.film_id " +
                 "INNER JOIN category ON film_category.category_id = category.category_id " +
                 "GROUP BY category.name ", con=connection)
        
df

,category,avg
0,Horror,112.482143
1,Comedy,115.827586
2,Sci-Fi,108.196721
3,Drama,120.838710
4,Foreign,121.698630
5,Classics,111.666667
6,Games,127.836066
7,New,111.126984
8,Travel,113.315789
9,Music,113.647059


Zadanie 10

In [13]:
df = pd.read_sql("SELECT category.name category, MAX(film.length) FROM film " +
                 "INNER JOIN film_category ON film.film_id = film_category.film_id " +
                 "INNER JOIN category ON film_category.category_id = category.category_id " +
                 "GROUP BY category.name", con=connection)
        
df

,category,max
0,Horror,181
1,Comedy,185
2,Sci-Fi,185
3,Drama,181
4,Foreign,184
5,Classics,184
6,Games,185
7,New,183
8,Travel,185
9,Music,185


Zadanie 11

In [14]:
df = pd.read_sql("SELECT * FROM film ORDER BY film.length DESC LIMIT 1" , con=connection)
df

,film_id,title,description,release_year,language_id,rental_duration,rental_rate,length,replacement_cost,rating,last_update,special_features,fulltext
0,141,Chicago North,A Fateful Yarn of a Mad Cow And a Waitress who...,2006,1,6,4.99,185,11.99,PG-13,2013-05-26 14:50:58.951,"[Deleted Scenes, Behind the Scenes]",'battl':15 'california':19 'chicago':1 'cow':9...


In [15]:
import main
main.film_in_category(2)

,title,name,name
0,Alter Victory,English,Animation
1,Anaconda Confessions,English,Animation
2,Argonauts Town,English,Animation
3,Bikini Borrowers,English,Animation
4,Blackout Private,English,Animation
...,...,...,...
61,Tracy Cider,English,Animation
62,Turn Star,English,Animation
63,Wait Cider,English,Animation
64,Watch Tracy,English,Animation


In [16]:
main.number_films_in_category(3)

,category,count
0,Children,60


In [24]:
main.number_film_by_length(100,130)

,length,count
0,100,12
1,101,7
2,102,11
3,103,9
4,104,6
5,105,6
6,106,6
7,107,10
8,108,5
9,109,7


In [19]:
main.client_from_city('Moscow')

,city,first_name,last_name
0,Moscow,Catherine,Campbell


In [20]:
main.avg_amount_by_length(100)

,length,avg
0,100,4.573214


In [21]:
main.client_by_sum_length(3000)

,first_name,last_name,sum
0,Phillip,Holm,3002
1,Kathleen,Adams,3003
2,Diana,Alexander,3007
3,Mario,Cheatham,3007
4,Stephanie,Mitchell,3008
...,...,...,...
301,Wesley,Bull,4808
302,Clara,Shaw,4808
303,Tammy,Sanders,5065
304,Eleanor,Hunt,5360


In [22]:
main.category_statistic_length('Horror')

,category,avg,sum,min,max
0,Horror,112.482143,6299,48,181
